In [89]:
import numpy as np
import pandas as pd
import glob
import os
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, MinMaxScaler
import joblib


In [102]:
states_trained = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\rentals.csv")
states = states_trained["State_Code"].unique()

# Load the new data
df = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Redfin Data\for_sale_homes.csv", index_col=False)

# Apply initial filtering on the DataFrame
df = df[(df["Bedrooms"] > 0) 
        & (df["Bedrooms"] < 6)  
        & (df["Bathrooms"] < 4)  
        & (df["Square Feet"] < 5000)    
        & (df["State"].notna())]

basic_features = ["Square Feet", "Bedrooms", "Bathrooms"]


In [100]:
#Get Median Income Data
median_income = pd.read_csv(r"C:\Users\mattl\OneDrive\Documents\reibrowser\Database\Rent Training Data\ACSDT5Y2022.B19013-Data.csv", skiprows=[1], na_values='-')

median_income['GEO_ID'] = median_income['GEO_ID'].astype(str)

# Extracting everything after "US" and storing it in a new column 'cbg_geoid'
median_income['cbg_geoid'] = median_income['GEO_ID'].str.extract('US(\d+)')
median_income['median_income'] = median_income['B19013_001E'].str.replace(',','+').str.replace('+','').str.replace('-','').astype(float)

median_income = median_income[["cbg_geoid", "median_income" ]]

print(df["MLS ID"].count())

df["cbg_geoid"] = df["cbg_geoid"].astype(str).str.split('.').str[0]
median_income["cbg_geoid"] = median_income["cbg_geoid"].astype(str)

df = df.merge(median_income, on='cbg_geoid', how="inner")

print(df["MLS ID"].count())

display(df["median_income"])

19424
0


Series([], Name: median_income, dtype: float64)

In [91]:

# Filter out rows where the state is not in the trained states
filtered_df = df[df["State_Code"].isin(states)]
filtered_out_count = len(df) - len(filtered_df)

# Show the count of predictions filtered out
print(f"Number of rows filtered out: {filtered_out_count}")

# Proceed with the prediction for the remaining rows
knn_features = ["Latitude", "Longitude"]
n_values = [1, 5, 10]

# Convert latitude and longitude from degrees to radians for haversine metric
new_data_rad = np.radians(filtered_df[knn_features].values)

# Loop over different values of n and use the saved models to predict rent benchmarks
for n in n_values:
    column_name = f'Rent_Benchmark_{n}_neighbors'
    
    # Load the trained model
    knn = joblib.load(fr'Models\knn_model_{n}_neighbors.pkl')
    
    # Predict the average rent for each point in the new data
    filtered_df[column_name] = knn.predict(new_data_rad)

# Display the new data with the predicted rent benchmarks

df = filtered_df

benchmark_features = [f'Rent_Benchmark_{n}_neighbors' for n in n_values]


Number of rows filtered out: 1


C:\Users\mattl\AppData\Local\Temp\ipykernel_24348\2476887341.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[column_name] = knn.predict(new_data_rad)
C:\Users\mattl\AppData\Local\Temp\ipykernel_24348\2476887341.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[column_name] = knn.predict(new_data_rad)
C:\Users\mattl\AppData\Local\Temp\ipykernel_24348\2476887341.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [76]:
columns_to_encode = ['Bedrooms', 'Bathrooms']

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap

# Fit and transform the data
encoded_features = encoder.fit_transform(df[columns_to_encode])

# Get feature names for the encoded columns
encoded_feature_names = encoder.get_feature_names_out(columns_to_encode)

# Create a DataFrame with the encoded features
encoded_df = pd.DataFrame(encoded_features, columns = encoded_feature_names)

# Concatenate the original DataFrame (excluding the original columns) with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

one_hot_features = encoded_feature_names.tolist()

In [ ]:
Selected_features = ['Bathrooms_2.0', 'Square Feet', 'Rent_Benchmark_1_neighbors', 'Rent_Benchmark_5_neighbors', 'Rent_Benchmark_10_neighbors', 'median_income^2', 'median_income Bedrooms_2', 'median_income Bathrooms_1.5', 'median_income Square Feet', 'median_income Bedrooms', 'median_income Rent_Benchmark_1_neighbors', 'median_income Rent_Benchmark_5_neighbors', 'median_income Rent_Benchmark_10_neighbors', 'Bedrooms_2 Bathrooms_2.0', 'Bedrooms_2 Square Feet', 'Bedrooms_2 Rent_Benchmark_1_neighbors', 'Bedrooms_2 Rent_Benchmark_5_neighbors', 'Bedrooms_3 Bathrooms_2.0', 'Bedrooms_3 Square Feet', 'Bedrooms_3 Rent_Benchmark_1_neighbors', 'Bedrooms_3 Rent_Benchmark_10_neighbors', 'Bedrooms_4 Bathrooms_2.0', 'Bedrooms_4 Square Feet', 'Bedrooms_4 Rent_Benchmark_1_neighbors', 'Bedrooms_4 Rent_Benchmark_10_neighbors', 'Bedrooms_5 Bathrooms_2.0', 'Bedrooms_5 Square Feet', 'Bedrooms_5 Rent_Benchmark_1_neighbors', 'Bedrooms_5 Rent_Benchmark_5_neighbors', 'Bedrooms_5 Rent_Benchmark_10_neighbors', 'Bathrooms_1.5 Bedrooms', 'Bathrooms_1.5 Rent_Benchmark_1_neighbors', 'Bathrooms_1.5 Rent_Benchmark_10_neighbors', 'Bathrooms_2.0^2', 'Bathrooms_2.0 Square Feet', 'Bathrooms_2.0 Bedrooms', 'Bathrooms_2.0 Bathrooms', 'Bathrooms_2.0 Rent_Benchmark_1_neighbors', 'Bathrooms_2.0 Rent_Benchmark_5_neighbors', 'Bathrooms_2.0 Rent_Benchmark_10_neighbors', 'Bathrooms_2.5 Square Feet', 'Bathrooms_2.5 Rent_Benchmark_1_neighbors', 'Bathrooms_2.5 Rent_Benchmark_5_neighbors', 'Bathrooms_2.5 Rent_Benchmark_10_neighbors', 'Bathrooms_3.0 Rent_Benchmark_1_neighbors', 'Bathrooms_3.0 Rent_Benchmark_5_neighbors', 'Bathrooms_3.0 Rent_Benchmark_10_neighbors', 'Square Feet^2', 'Square Feet Bedrooms', 'Square Feet Bathrooms', 'Square Feet Rent_Benchmark_1_neighbors', 'Square Feet Rent_Benchmark_5_neighbors', 'Square Feet Rent_Benchmark_10_neighbors', 'Bedrooms Bathrooms', 'Bedrooms Rent_Benchmark_1_neighbors', 'Bedrooms Rent_Benchmark_5_neighbors', 'Bedrooms Rent_Benchmark_10_neighbors', 'Bathrooms Rent_Benchmark_1_neighbors', 'Rent_Benchmark_1_neighbors^2', 'Rent_Benchmark_1_neighbors Rent_Benchmark_5_neighbors', 'Rent_Benchmark_1_neighbors Rent_Benchmark_10_neighbors', 'Rent_Benchmark_5_neighbors^2', 'Rent_Benchmark_5_neighbors Rent_Benchmark_10_neighbors', 'Rent_Benchmark_10_neighbors^2']
